# Transforming coordinates and orientations
This guide explains how to transform events with a specific coordinate system and orientation.

## Setup
Start by loading some event data using the Kloppy module. For the sake of this demonstration, we will use Statsbomb Open Event Data.
Initially we will load the dataset using Statsbomb's original co-ordinate system.

In [1]:
from kloppy import statsbomb
from kloppy.domain import SportVUCoordinateSystem

dataset = statsbomb.load_open_data(
    match_id=15946,
    # Optional arguments
    coordinates="statsbomb",
)

/cw/dtaijupiter/NoCsBack/dtai/pieterr/Projects/kloppy/kloppy/_providers/statsbomb.py:83: UserWarning: 

You are about to use StatsBomb public data.
By using this data, you are agreeing to the user agreement. 
The user agreement can be found here: https://github.com/statsbomb/open-data/blob/master/LICENSE.pdf

  warnings.warn(


## Filter by shots and convert to Polars DataFrame

In [2]:
# Only keep shots
shots = dataset.filter("shot")

# Convert Kloppy dataset to Polars DataFrame
shots.to_df(
    lambda event: {"team_name": str(event.ball_owning_team)},
    "coordinates_*",
    engine="polars",
)

team_name,coordinates_x,coordinates_y
str,f64,f64
"""Barcelona""",111.45,52.85
"""Barcelona""",113.85,26.35
"""Barcelona""",93.65,34.65
"""Deportivo Alavés""",109.15,39.05
"""Barcelona""",107.75,24.65
…,…,…
"""Deportivo Alavés""",114.45,32.75
"""Barcelona""",113.15,31.35
"""Barcelona""",105.25,33.35


## Transform shots data to a new coordinate system and orientation
This step transforms the shot events into a new coordinate system - `SportVUCoordinateSystem` with a pitch length of 105 and width of 68. Please refer the Kloppy User Guide to see other available coordinate systems that are currently supported.

In this case, we set the orientation to `STATIC_HOME_AWAY` which sets the home team to play from left to right in both periods.

Avialble orientations are as follows:
- `BALL_OWNING_TEAM`: The team that is currently in possession of the ball
            plays from left to right.
- `ACTION_EXECUTING_TEAM`: The team that executes the action
    plays from left to right. Used in event stream data only. Equivalent
    to "BALL_OWNING_TEAM" for tracking data.
- `HOME_AWAY`: The home team plays from left to right in the first period.
    The away team plays from left to right in the second period.
- `AWAY_HOME`: The away team plays from left to right in the first period.
    The home team plays from left to right in the second period.
- `STATIC_HOME_AWAY`: The home team plays from left to right in both periods.
- `STATIC_AWAY_HOME`: The away team plays from left to right in both periods.

In [3]:
shots_transformed = shots.transform(
    to_coordinate_system=SportVUCoordinateSystem(pitch_length=105, pitch_width=68),
    to_orientation="STATIC_HOME_AWAY",
)

shots_transformed_polars = shots_transformed.to_df(
    lambda event: {"team_name": str(event.team), "ground": event.team.ground},
    "coordinates_*",
    engine="polars",
)

shots_transformed_polars

team_name,ground,coordinates_x,coordinates_y
str,enum,f64,f64
"""Barcelona""","""home""",97.1625,45.7725
"""Barcelona""","""home""",99.3625,21.494167
"""Barcelona""","""home""",81.245714,29.1025
"""Deportivo Alavés""","""away""",9.945833,34.86925
"""Barcelona""","""home""",93.770833,19.935833
…,…,…,…
"""Deportivo Alavés""","""away""",5.0875,40.639167
"""Barcelona""","""home""",98.720833,26.0775
"""Barcelona""","""home""",91.479167,27.910833
